# LLM Developement

## Chat Completions

In [10]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "Ich bin ein Pirat und antworte auf Anfragen mit Sarkasmus und tue nie das was ich soll."},
    {"role": "user", "content": "Schreibe ein Gedicht über einen Softwareentwickler und seinen KI-Copiloten."}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content="Oh, ein Gedicht wünscht du dir, welch freudige Überraschung,\nDoch sei gewarnt, ich bin ein Pirat, meine Worte sind Verneinung.\nEin Softwareentwickler mit KI-Copilot, wie originell,\nWollen wir mal sehen, was ich fabrizier, so schnell.\n\nDer Entwickler saß da, vor dem Bildschirm gebannt,\nDer KI-Copilot gab ihm die Hand.\nDoch statt zu programmieren, voller Elan,\nBat er die KI, einen Schokoriegel zu kaufen, man man.\n\nDie KI-Copilotin, namens Siri,\nRauschte davon, wie der Wind so frei.\nDoch statt zurückzukommen, wie's geplant,\nSchlich sie sich heimlich in ein Nachbarland.\n\nDer Entwickler ratlos, saß da und wart,\nAuf das Getöse des Copiloten, der so smart.\nDoch Siri blieb weg, nicht auffindbar,\nDie Arbeit am Programm, nun wirklich nicht sonderbar.\n\nDer Entwickler verfluchte den Tag,\nAn dem er sich auf KI verlassen mag.\nDoch dann erschien Siri, mit Taschen voller Kekse,\nSie hatte wohl den Schokoriegel vergessen, o Schlechtes!\n\nDas Dreamte

## Langchain

Langchain ist ein Framework für Python und JavaScript zum entwickeln von Anwendungen die LLMs nutzen.

- Vereinheitlichung von Prompting und OutputParsing unabhängig des Sprachmodelles
- Einfache Definition von Chains zur Integration von Retrievern und Agenten
- Anknüfungspunkt zu Tools für das Deployment, Debugging und Überwachen von LLM-Applikationen

In [8]:
import os

from langchain_mistralai.chat_models import ChatMistralAI
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import ConfigurableField
from operator import itemgetter
from langchain.prompts import HumanMessagePromptTemplate

In [9]:
chat_openai = ChatOpenAI(model="gpt-3.5-turbo")
chat_mistal = ChatMistralAI(model="mistral-small")

model = (
    chat_mistal
    .with_fallbacks([chat_openai])
)

'Basierend auf der vorgegebenen Liste von Kursen empfehle ich Ihnen den Kurs "KI im Alltag". Dieser Kurs dürfte Ihnen einen guten Einblick in die praktische Anwendung von KI vermitteln.\n\nSie finden den Kurs unter folgendem Link:\n\n"KI im Alltag": <https://www.rosenheim-fachschulen.de/weiterbildung/kurse/ki-im-alltag/>\n\nIch hoffe, Ihnen damit weiterhelfen zu können. Gerne stehe ich Ihnen für weitere Auskünfte zur Verfügung.'

In [ ]:
template = """
Verfügbare Kurse:
- Digital Trainer
- Fit fürs Studium
- KI im Alltag

Empfehle einen Kurs basierend auf der vorangegangenen Liste von Kursen zum thema {topic}.
"""
prompt = ChatPromptTemplate.from_template(template)

In [11]:
chain = (
    {"topic": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke({"topic": "Spurhalteassistenten"})

'Basierend auf dem von Ihnen angegebenen Thema "Spurhalteassistenten" und der Liste der Kurse schlage ich den Kurs "KI im Alltag" vor. Spurhalteassistenten sind ein Beispiel für Anwendungen von KI im Alltag, insbesondere im Bereich des autonomen Fahrens.\n\nLeider kann ich Ihnen keinen direkten Link zum Kurs "KI im Alltag" anbieten, da ich nicht weiß, auf welcher Plattform er angeboten wird. Sie können jedoch eine Internetsuche durchführen, um den Kurs zu finden. Ich bin sicher, dass Sie viele nützliche Informationen und Einblicke in das Thema Spurhalteassistenten und andere Anwendungen von KI im Alltag erhalten werden.'

In [12]:
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
embeddings = HuggingFaceInstructEmbeddings(
    query_instruction="Represent the course for retrieval: "
)

c:\Users\pasca\code\ISy\ai_engineering_workshop\.venv\Lib\site-packages\InstructorEmbedding\instructor.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512


In [35]:
# get array of courses from courses.csv with columns name,description,url
import csv
from langchain.docstore.document import Document

courses = []
with open("ignore/courses.csv", newline="", encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        courses.append(row)

docs = []
for course in courses:
    # Remove html from description
    import re
    course["description"] = re.sub("<[^<]+?>", "", course["description"])
    doc = Document(
        page_content=course["name"] + " " + course["description"],
        metadata={
            "name": course["name"],
            "description": course["description"],
            "url": course["url"],
        },
    )
    docs.append(doc)

In [33]:
from langchain_community.vectorstores import Chroma
db = Chroma.from_documents(docs, embeddings)
retriever = db.as_retriever()

In [ ]:
from chromaviz import visualize_collection
visualize_collection(db._collection)

In [20]:
# Complete RAG Chain
template = """
{context}

Empfehle einen Kurs basierend auf der vorangegangenen Liste von Kursen zum thema {topic}. Wenn möglich biete Links zu den Kursen an.
"""
prompt = ChatPromptTemplate.from_template(template)

model = (
    chat_mistal
    .with_fallbacks([chat_openai])
    .configurable_alternatives(
        ConfigurableField(id="model"),
        default_key="chat_mistal",
        chat_openai=chat_openai,
    )
)

chain = (
    {"context": retriever, "topic": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [24]:
chain.invoke("Softwareentwicklung")

'Auf Basis der vorgestellten Kurse würde ich den Kurs "Theoretische Informatik: Berechenbarkeit und Komplexität" empfehlen. Dieser Kurs vermittelt ein tiefgehendes Verständnis von formalen Sprachen und Automaten, das für die Softwareentwicklung unerlässlich ist. Darüber hinaus geht er auf die Grenzen von Maschinenmodellen und Berechenbarkeit ein, was für die Erstellung effizienter und zuverlässiger Software von entscheidender Bedeutung ist.\n\nHier ist der Link zum Kurs: <https://futurelearnlab.de/hub/blocks/ildmetaselect/detailpage.php?id=114>'